# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [123]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [124]:
stocks=pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [125]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data =requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2200739911147,
 'week52high': 140.24,
 'week52low': 55.91,
 'week52change': 0.8719458933597886,
 'sharesOutstanding': 17406171420,
 'float': 0,
 'avg10Volume': 101668245,
 'avg30Volume': 99859603,
 'day200MovingAvg': 112.49,
 'day50MovingAvg': 124.48,
 'employees': 0,
 'ttmEPS': 3.34,
 'ttmDividendRate': 0.8364791566179018,
 'dividendYield': 0.006353563346785369,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-29',
 'nextEarningsDate': '0',
 'peRatio': 37.93347047755254,
 'beta': 1.1706369416413662,
 'maxChangePercent': 48.28587423723624,
 'year5ChangePercent': 4.067532019044992,
 'year2ChangePercent': 2.187612019522588,
 'year1ChangePercent': 0.8756953596727327,
 'ytdChangePercent': 0.7629397982274565,
 'month6ChangePercent': 0.462729619095283,
 'month3ChangePercent': 0.1422842951540848,
 'month1ChangePercent': 0.0629910568268863,
 'day30ChangePercent': 0.06458708438414008,
 'day5ChangePercent': 0.05022004447592658}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [126]:
data['year1ChangePercent']

0.8756953596727327

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [127]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [128]:
final_dataframe=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
                
        ],
            index=my_columns
            ),
            ignore_index= True
        )
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,121.670,0.419235,N/A
1,AAL,17.335,-0.398946,N/A
2,AAP,164.310,0.0594531,N/A
3,AAPL,128.990,0.852492,N/A
4,ABBV,106.450,0.24345,N/A
...,...,...,...,...
500,YUM,112.950,0.086367,N/A
501,ZBH,156.640,0.000233018,N/A
502,ZBRA,390.200,0.478726,N/A
503,ZION,43.110,-0.154047,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [129]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(drop=True,inplace=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,40.390,2.27352,N/A
1,NVDA,536.530,1.39174,N/A
2,AMD,101.240,1.32592,N/A
3,LB,39.670,1.22475,N/A
4,PYPL,244.770,1.15314,N/A
5,ALB,142.350,1.13459,N/A
6,NOW,553.560,0.965439,N/A
7,ALGN,543.560,0.948699,N/A
8,FCX,25.761,0.894853,N/A
9,AMZN,3281.900,0.870082,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [130]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input('Enter the size of your portfolio:')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number /n Please try again')
        portfolio_size=input('Enter the size of your portfolio:')
portfolio_input()
portfolio_size

Enter the size of your portfolio:10000000


'10000000'

In [131]:
position_size=float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe    

/usr/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,40.390,2.27352,4951
1,NVDA,536.530,1.39174,372
2,AMD,101.240,1.32592,1975
3,LB,39.670,1.22475,5041
4,PYPL,244.770,1.15314,817
5,ALB,142.350,1.13459,1404
6,NOW,553.560,0.965439,361
7,ALGN,543.560,0.948699,367
8,FCX,25.761,0.894853,7763
9,AMZN,3281.900,0.870082,60


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [132]:
hqm_columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe=pd.DataFrame(columns=hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe=hqm_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
                
        ],
            index=hqm_columns
            ),
            ignore_index= True
        )
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,123.680,N/A,0.404233,N/A,0.363173,N/A,0.180599,N/A,0.0649816,N/A,N/A
1,AAL,16.962,N/A,-0.405605,N/A,-0.0104522,N/A,0.229012,N/A,0.322912,N/A,N/A
2,AAP,168.620,N/A,0.0596164,N/A,0.13746,N/A,0.0459537,N/A,0.0681477,N/A,N/A
3,AAPL,134.440,N/A,0.883718,N/A,0.460032,N/A,0.146274,N/A,0.0650725,N/A,N/A
4,ABBV,108.900,N/A,0.241106,N/A,0.121539,N/A,0.186553,N/A,0.0645213,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.130,N/A,0.0852337,N/A,0.184037,N/A,0.144611,N/A,0.030251,N/A,N/A
501,ZBH,150.200,N/A,0.000229127,N/A,0.144375,N/A,0.0220441,N/A,-0.00935875,N/A,N/A
502,ZBRA,381.200,N/A,0.492159,N/A,0.417218,N/A,0.506363,N/A,0.0932438,N/A,N/A
503,ZION,43.480,N/A,-0.153028,N/A,0.160281,N/A,0.33302,N/A,0.0552757,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [133]:
#hqm_dataframe=hqm_dataframe[:300]
time_periods=[
                'One-Year',
                'Six-Month',
                'Three-Month',  
                'One-Month'
             ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
            
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col= f'{time_period} Price Return'
        percentile_col=f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe
        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,123.680,N/A,0.404233,0.869307,0.363173,0.815842,0.180599,0.649505,0.0649816,0.730693,N/A
1,AAL,16.962,N/A,-0.405605,0.0277228,-0.0104522,0.120792,0.229012,0.770297,0.322912,0.992079,N/A
2,AAP,168.620,N/A,0.0596164,0.469307,0.13746,0.380198,0.0459537,0.285149,0.0681477,0.744554,N/A
3,AAPL,134.440,N/A,0.883718,0.982178,0.460032,0.89703,0.146274,0.564356,0.0650725,0.732673,N/A
4,ABBV,108.900,N/A,0.241106,0.732673,0.121539,0.342574,0.186553,0.673267,0.0645213,0.728713,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.130,N/A,0.0852337,0.514851,0.184037,0.50099,0.144611,0.550495,0.030251,0.582178,N/A
501,ZBH,150.200,N/A,0.000229127,0.382178,0.144375,0.39802,0.0220441,0.223762,-0.00935875,0.344554,N/A
502,ZBRA,381.200,N/A,0.492159,0.906931,0.417218,0.861386,0.506363,0.976238,0.0932438,0.809901,N/A
503,ZION,43.480,N/A,-0.153028,0.164356,0.160281,0.415842,0.33302,0.908911,0.0552757,0.69703,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [134]:
from statistics import mean

for row in hqm_dataframe.index:
    
    momentum_percentiles=[]
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score']=mean(momentum_percentiles)
hqm_dataframe    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,123.680,N/A,0.404233,0.869307,0.363173,0.815842,0.180599,0.649505,0.0649816,0.730693,0.766337
1,AAL,16.962,N/A,-0.405605,0.0277228,-0.0104522,0.120792,0.229012,0.770297,0.322912,0.992079,0.477723
2,AAP,168.620,N/A,0.0596164,0.469307,0.13746,0.380198,0.0459537,0.285149,0.0681477,0.744554,0.469802
3,AAPL,134.440,N/A,0.883718,0.982178,0.460032,0.89703,0.146274,0.564356,0.0650725,0.732673,0.794059
4,ABBV,108.900,N/A,0.241106,0.732673,0.121539,0.342574,0.186553,0.673267,0.0645213,0.728713,0.619307
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.130,N/A,0.0852337,0.514851,0.184037,0.50099,0.144611,0.550495,0.030251,0.582178,0.537129
501,ZBH,150.200,N/A,0.000229127,0.382178,0.144375,0.39802,0.0220441,0.223762,-0.00935875,0.344554,0.337129
502,ZBRA,381.200,N/A,0.492159,0.906931,0.417218,0.861386,0.506363,0.976238,0.0932438,0.809901,0.888614
503,ZION,43.480,N/A,-0.153028,0.164356,0.160281,0.415842,0.33302,0.908911,0.0552757,0.69703,0.546535


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [135]:
hqm_dataframe.sort_values('HQM Score',ascending=False,inplace=True)
hqm_dataframe =hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True,inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.511,N/A,0.907531,0.984158,1.31206,0.99802,0.472753,0.962376,0.164285,0.908911,0.963366
1,ALGN,531.940,N/A,0.933058,0.986139,0.980845,0.992079,0.591593,0.984158,0.135523,0.881188,0.960891
2,TWTR,54.885,N/A,0.770306,0.962376,0.584586,0.946535,0.365272,0.924752,0.270756,0.982178,0.95396
3,LRCX,514.370,N/A,0.794037,0.966337,0.637311,0.960396,0.624683,0.988119,0.127896,0.873267,0.94703
4,XLNX,156.350,N/A,0.627972,0.946535,0.647152,0.962376,0.505357,0.974257,0.156165,0.89901,0.945545
5,AMAT,88.490,N/A,0.498787,0.914851,0.529006,0.938614,0.581609,0.982178,0.201903,0.944554,0.94505
6,LB,39.770,N/A,1.21822,0.994059,1.4014,1,0.303147,0.875248,0.114872,0.849505,0.929703
7,AMD,101.290,N/A,1.33594,0.99604,0.782083,0.982178,0.27108,0.833663,0.160967,0.90297,0.928713
8,PYPL,240.780,N/A,1.10899,0.990099,0.436629,0.881188,0.273365,0.837624,0.204326,0.948515,0.914356
9,ALXN,159.040,N/A,0.43923,0.889109,0.410818,0.853465,0.368873,0.926733,0.244863,0.972277,0.910396


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [136]:
portfolio_input()

Enter the size of your portfolio:10000000


In [137]:
position_size=float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
hqm_dataframe    

/usr/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FCX,25.511,4951,0.907531,0.984158,1.31206,0.99802,0.472753,0.962376,0.164285,0.908911,0.963366
1,ALGN,531.940,372,0.933058,0.986139,0.980845,0.992079,0.591593,0.984158,0.135523,0.881188,0.960891
2,TWTR,54.885,1975,0.770306,0.962376,0.584586,0.946535,0.365272,0.924752,0.270756,0.982178,0.95396
3,LRCX,514.370,5041,0.794037,0.966337,0.637311,0.960396,0.624683,0.988119,0.127896,0.873267,0.94703
4,XLNX,156.350,817,0.627972,0.946535,0.647152,0.962376,0.505357,0.974257,0.156165,0.89901,0.945545
5,AMAT,88.490,1404,0.498787,0.914851,0.529006,0.938614,0.581609,0.982178,0.201903,0.944554,0.94505
6,LB,39.770,361,1.21822,0.994059,1.4014,1,0.303147,0.875248,0.114872,0.849505,0.929703
7,AMD,101.290,367,1.33594,0.99604,0.782083,0.982178,0.27108,0.833663,0.160967,0.90297,0.928713
8,PYPL,240.780,7763,1.10899,0.990099,0.436629,0.881188,0.273365,0.837624,0.204326,0.948515,0.914356
9,ALXN,159.040,60,0.43923,0.889109,0.410818,0.853465,0.368873,0.926733,0.244863,0.972277,0.910396


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [138]:
writer= pd.ExcelWriter('momentum_stratergy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name='Momentum_Stratergy',index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [139]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [140]:
column_formats={
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['One-Year Price Return',percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}
for column in column_formats.keys():
    writer.sheets['Momentum_Stratergy'].set_column(f'{column}:{column}',22,column_formats[column][1])
    writer.sheets['Momentum_Stratergy'].write(f'{column}1',column_formats[column][0],column_formats[column][1])



## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [141]:
writer.save()    